### ETL initialization with pandas and numpy

In [1]:
import pandas as pd
from collections import OrderedDict
import numpy as np
pd.set_option('display.max_columns', None)
pd.set_option('display.expand_frame_repr', False)
pd.set_option('max_colwidth', None)
pd.set_option('mode.chained_assignment',None)

### pandas read CSV file: `firm_trades.csv`
##### first 3 rows are header values (ORDER, ROUTE, FILL) with respective column names

In [2]:
df = pd.read_csv('./data/firm_trades.csv')
df.head()

,ORDER,Order Number,Ticker,Exchange,Security Name,Instructions,Side,TIF,Status,Amount,Working Amount,Filled Amount,Average Price,Broker,BBGID,Create Date,Create Time,ISIN,SEDOL,Trader Name,Ticker + Exchange,OCC_Symbol,Yellow Key,Asset Class,As Of Date,As Of Time,CUSIP,Order Type,Trader UUID,Exec Instructions,Order Ref ID,Handling Instr,Account,Last Fill Date,Trader Notes,Limit Price
0,ROUTE,Order Number,Ticker,Exchange,Security Name,Route Number,Side,Route Status,Route TIF,Route Type,Routed Amount,Route Filled Amount,Route Avg Price,Route Date,Route Time,Route As Of Date,Route As Of Time,Broker,Execution Broker,CUSIP,ISIN,SEDOL,Route Comm Amount,Route Comm Rate,Exec Instr,Account,Instructions,Yellow Key,Asset Class,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,FILL,Order Number,Ticker,Exchange,Security Name,Route Number,Side,Route Status,Fill Amount,Fill Price,Execution Type,Exec Seq Num,Prev Exec Seq Num,Route Date,Route Time,Fill As Of Date,Fill As Of Time,Broker,Execution Broker,CUSIP,ISIN,SEDOL,Trader Name,Account,Yellow Key,Trader UUID,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,ORDER,3520287,GUT,YA,GUT CYP,NaN,SL,DAY,Part-filled,7804618.811,0,1382802,141.2247,ZYVA,OOT00K3W13Y1,06/07/22,4:41:25,TO00OZGI7393,OZGI739,NYVPR QNEEL,GUT YA,NaN,Rdhvgl,Rdhvgl,06/07/22,20:00:07,A.N.,YZG,30773047,NaN,NaN,DESK,NaN,06/07/22,NaN,140.0
3,ROUTE,3520287,GUT,YA,GUT CYP,1,SL,Cancel,DAY,LMT,1382802,1382802,141.2246721,06/07/22,4:41:25,06/07/22,11:37:09,ZYVA,NaN,A.N.,TO00OZGI7393,OZGI739,0,0,NaN,NaN,NaN,Rdhvgl,Rdhvgl,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,FILL,3520287,GUT,YA,GUT CYP,1,SL,Part-filled,2169,140.5,Exec-Trade,1,0,06/07/22,4:41:25,06/07/22,4:44:07,ZYVA,NaN,A.N.,TO00OZGI7393,OZGI739,NYVPR QNEEL,NaN,Rdhvgl,184447706,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Column names changes for Sqlite column stanard

In [3]:
cols_df = df[:2].T.reset_index()
cols_df.columns = cols_df[:1].values.tolist()[0]
cols_df = cols_df[1:]
cols_df = cols_df.applymap(lambda _c: str(_c).lower().replace(' ', '_'))
cols_df

,ORDER,ROUTE,FILL
1,order_number,order_number,order_number
2,ticker,ticker,ticker
3,exchange,exchange,exchange
4,security_name,security_name,security_name
5,instructions,route_number,route_number
6,side,side,side
7,tif,route_status,route_status
8,status,route_tif,fill_amount
9,amount,route_type,fill_price
10,working_amount,routed_amount,execution_type


### Separating data based on the row types ('ORDER', 'FILL', 'ROUTE')
##### also applying Datatypes for each columns

In [4]:
extracted_df = {}
df_dtypes = {
    'ROUTE': {'order_id': int, 'route_number': int, 'routed_amount': int, 'route_filled_amount': int, 'route_avg_price': float, 'route_comm_amount': int, 'route_comm_rate': int},
    'FILL': {'order_id': int, 'route_number': int, 'fill_amount': int, 'fill_price': float, 'exec_seq_num': int, 'prev_exec_seq_num': int, 'trader_uuid': int},
    'ORDER': {'order_id': int, 'amount': float, 'working_amount': int, 'filled_amount': int, 'average_price': float, 'trader_uuid': int, 'limit_price': float}
}

for c in ['ORDER', 'FILL', 'ROUTE']:
    r_df = df[2:]
    r_df.set_index(r_df.columns[0], inplace=True)
    r_df.columns = cols_df[c].values.tolist()
    r_df.rename(columns = {'order_number': 'order_id'}, inplace = True)
    r_df = r_df[r_df.index == c]
    r_df.reset_index(inplace=True)
    del r_df["ORDER"]
    r_df = r_df.astype(df_dtypes[c])
    extracted_df.setdefault(c, r_df)

def filter_df(_df):
    return _df[filter(lambda _c: str(_c) != 'nan', _df.columns)]

### Separated df are stored in different variables

In [5]:
order_df = filter_df(extracted_df['ORDER'])
route_df = filter_df(extracted_df['ROUTE'])
fill_df = filter_df(extracted_df['FILL'])

### Creating datetime columns from date, time columns with timezone `America/New_York`

In [6]:
order_df['create_date_time'] = pd.to_datetime(order_df["create_date"] + ' ' + order_df["create_time"]).dt.tz_localize('America/New_York').apply(lambda d: d.isoformat())
order_df.drop(["create_date", "create_time"], axis=1, inplace=True)

fill_df['fill_as_of_date_time'] = pd.to_datetime(fill_df["fill_as_of_date"] + ' ' + fill_df["fill_as_of_time"]).dt.tz_localize('America/New_York').apply(lambda d: d.isoformat())
fill_df.drop(["fill_as_of_date", "fill_as_of_time"], axis=1, inplace=True)

route_df['route_as_of_date_time'] = pd.to_datetime(route_df["route_as_of_date"] + ' ' + route_df["route_as_of_time"]).dt.tz_localize('America/New_York').apply(lambda d: d.isoformat())
route_df.drop(["route_as_of_date", "route_as_of_time"], axis=1, inplace=True)

### Creating `Sqlite` engine for database `my_lite_store.db`

In [7]:
from sqlalchemy import create_engine

disk_engine = create_engine('sqlite:////home/absolute/jupyter-lab/app/my_lite_store.db')

### Creating and Inserting tables for each row types ('ORDER', 'FILL', 'ROUTE')

In [8]:
order_df.to_sql('order_table', disk_engine, if_exists='append', index=False)
route_df.to_sql('route_table', disk_engine, if_exists='append', index=False)
fill_df.to_sql('fill_table', disk_engine, if_exists='append', index=False)

5106

### Querying each table to verify data insertion

In [143]:
_o_df = pd.read_sql_query('SELECT * FROM order_table where order_id = 3520557', disk_engine)
_o_df.head()

,order_id,ticker,exchange,security_name,instructions,side,tif,status,amount,working_amount,filled_amount,average_price,broker,bbgid,isin,sedol,trader_name,ticker_+_exchange,occ_symbol,yellow_key,asset_class,as_of_date,as_of_time,cusip,order_type,trader_uuid,exec_instructions,order_ref_id,handling_instr,account,last_fill_date,trader_notes,limit_price,create_date_time
0,3520557,OBJY,HF,OBJYREB PBEC,None,SL,DAY,Part-filled,243814.7589,0,72128,11.1815,OGVT,OOT00M6OW689,HF10258C1021,OCT5S19,QNAVRY PNAABA,OBJY HF,None,Rdhvgl,Rdhvgl,06/07/22,16:02:16,10258C10,YZG,20076884,None,None,ANY,APDBTIG,06/07/22,None,11.0,2022-06-07 10:09:34.000000


In [161]:
_f_df = pd.read_sql_query('SELECT * FROM fill_table where order_id = 3520557', disk_engine)
_f_df.head()

,order_id,ticker,exchange,security_name,instructions,side,tif,status,amount,working_amount,filled_amount,average_price,broker,bbgid,isin,sedol,trader_name,ticker_+_exchange,occ_symbol,yellow_key,asset_class,as_of_date,as_of_time,cusip,order_type,trader_uuid,exec_instructions,order_ref_id,handling_instr,account,last_fill_date,trader_notes,limit_price,create_date_time
0,3520680,FNOE,HF,FNOER PBEC,None,SL,DAY,Filled,3.499935e+06,0,3499935,7.560000,AZJB,OOT005JDIIU4,HF78573Z1045,OYYUU27,QNAVRY PNAABA,FNOE HF,None,Rdhvgl,Rdhvgl,06/07/22,16:00:10,78573Z10,ZXG,20076884,None,None,MAN,SWAP,06/07/22,None,0.00,2022-06-07 13:28:43.000000
1,3520681,FNOE,HF,FNOER PBEC,None,BY,DAY,Filled,3.499935e+06,0,3499935,7.560000,AZJB,OOT005JDIIU4,HF78573Z1045,OYYUU27,QNAVRY PNAABA,FNOE HF,None,Rdhvgl,Rdhvgl,06/07/22,16:00:11,78573Z10,ZXG,20076884,None,None,MAN,CASH,06/07/22,None,0.00,2022-06-07 13:29:07.000000
2,3520846,OXYA,HF,VAIRFPB FRAVBE YBNA RGS,None,BY,DAY,Filled,7.804619e+06,0,7804618,21.248998,PBJR,OOT001X94A28,HF46138T5080,OM03Y33,QNAVRY PNAABA,OXYA HF,None,Rdhvgl,Rdhvgl,06/07/22,16:00:32,46138T50,YZG,20076884,None,None,ANY,None,06/07/22,None,21.25,2022-06-07 15:45:42.000000
3,3520854,J,HF,JNLSNVE VAP- PYNFF N,None,BY,DAY,Filled,5.208372e+03,0,5208,54.811400,WRSN,OOT001O17ZI2,HF94419Y1017,ODKMC64,ZVPUNRY PNCREBAVF,J HF,None,Rdhvgl,Rdhvgl,06/07/22,15:55:29,94419Y10,YZG,30925559,None,None,ANY,None,06/07/22,None,54.99,2022-06-07 15:55:14.000000


In [ ]:
_r_df = pd.read_sql_query('SELECT * FROM route_table where order_number = 3520557', disk_engine)
_r_df.head()

['2022-06-07T04:41:25-04:00',
 '2022-06-07T09:23:12-04:00',
 '2022-06-07T09:26:37-04:00',
 '2022-06-07T09:28:37-04:00',
 '2022-06-07T10:09:34-04:00',
 '2022-06-07T11:24:51-04:00',
 '2022-06-07T12:49:16-04:00',
 '2022-06-07T12:57:01-04:00',
 '2022-06-07T13:28:43-04:00',
 '2022-06-07T13:29:07-04:00',
 '2022-06-07T14:25:17-04:00',
 '2022-06-07T15:45:42-04:00',
 '2022-06-07T15:55:14-04:00']

In [25]:
order_df['create_date_time'] = map(lambda d: d.isoformat(), pd.to_datetime(order_df["create_date"] + ' ' + order_df["create_time"]).dt.tz_localize('America/New_York'))

SyntaxError: unmatched ')' (1356618273.py, line 1)

In [30]:
pd.to_datetime(order_df["create_date"] + ' ' + order_df["create_time"]).dt.tz_localize('America/New_York')

0     2022-06-07 04:41:25-04:00
1     2022-06-07 09:23:12-04:00
2     2022-06-07 09:26:37-04:00
3     2022-06-07 09:28:37-04:00
4     2022-06-07 10:09:34-04:00
5     2022-06-07 11:24:51-04:00
6     2022-06-07 12:49:16-04:00
7     2022-06-07 12:57:01-04:00
8     2022-06-07 13:28:43-04:00
9     2022-06-07 13:29:07-04:00
10    2022-06-07 14:25:17-04:00
11    2022-06-07 15:45:42-04:00
12    2022-06-07 15:55:14-04:00
dtype: object

In [39]:
pd.to_datetime(order_df["create_date"] + ' ' + order_df["create_time"]).dt.tz_localize('America/New_York').apply(lambda d: d.isoformat())

0     2022-06-07T04:41:25-04:00
1     2022-06-07T09:23:12-04:00
2     2022-06-07T09:26:37-04:00
3     2022-06-07T09:28:37-04:00
4     2022-06-07T10:09:34-04:00
5     2022-06-07T11:24:51-04:00
6     2022-06-07T12:49:16-04:00
7     2022-06-07T12:57:01-04:00
8     2022-06-07T13:28:43-04:00
9     2022-06-07T13:29:07-04:00
10    2022-06-07T14:25:17-04:00
11    2022-06-07T15:45:42-04:00
12    2022-06-07T15:55:14-04:00
dtype: object